In [1]:
from markdown_parser.custom_ast import Text, Break, Heading, Listing, Table, Ast, Item, Paragraph
from markdown_parser import md_to_ast
from custom_renderer import CustomRenderer, Text, Break, Heading, Listing, Table, Ast, Item, Paragraph
import mistune_v2

from pydantic import BaseModel, validator
import pandas as pd
import more_itertools
from enum import Enum
from typing import Optional
import re
import ipywidgets as ipw
from collections import defaultdict
from dataclasses import dataclass

In [2]:
custom_ast = mistune_v2.create_markdown(
     plugins=['table', 'strikethrough'], 
    renderer=CustomRenderer()
)

In [3]:
def reddit_link(row):
    return f"https://reddit.com/{row['reddit_id']}"

In [4]:
data_full = pd.read_csv('adverts.csv').drop_duplicates(subset=['full_text'], keep='last')

In [5]:
data_full['mistune'] = data_full['full_text'].apply(lambda text: Ast(children=custom_ast(text)))
data_full['misaka'] = data_full['full_text'].apply(md_to_ast)


In [6]:
def clean_table(table: Table):
    ok_rows = []
    for row in table.rows:
        if any(row):
            ok_rows.append(row)
    
    return Table(rows=ok_rows)

def extract_tables(item: Item):
#     print("extracting tables from", item)
#     print("AST", isinstance(item, Ast))
    if isinstance(item, Table):
        return [item]
    elif (
        isinstance(item, Ast) or
        isinstance(item, Listing) or 
        isinstance(item, Paragraph)
    ):
        extracted_tables_lists = [extract_tables(child) for child in item.children]
        tables = [clean_table(t) for tables in extracted_tables_lists for t in tables if tables is not None]
#         print("Found in children:", tables)
        return tables
    else:
        return []
    

In [13]:
for i, row in data_full.iterrows():
#     print("MISTUNE")
    tables_mistune = extract_tables(row['mistune'])
    
#     print()
#     print("MISAKA")
    tables_misaka = extract_tables(row['misaka'])
#     print()
    
    if len(tables_mistune) < len(tables_misaka):
        print(i, f"mistune {len(tables_mistune)}; misaka {len(tables_misaka)}")
        print(reddit_link(row))
#         print(row["full_text"])
        

969 mistune 0; misaka 1
https://reddit.com/nemd3g
1156 mistune 0; misaka 1
https://reddit.com/o3ciaz
3306 mistune 0; misaka 1
https://reddit.com/o28vji
3871 mistune 0; misaka 1
https://reddit.com/nt7zxh
4528 mistune 0; misaka 1
https://reddit.com/opk4pc
6271 mistune 0; misaka 1
https://reddit.com/oue4oc
7370 mistune 2; misaka 3
https://reddit.com/osuczy
7723 mistune 0; misaka 1
https://reddit.com/ojjzbz
9726 mistune 0; misaka 1
https://reddit.com/oebs21


In [11]:
row['misaka']

Ast(children=[Paragraph(children=[Text(text='timestamp: https://imgur.com/a/4vLfWjI', striked=False)], type='Paragraph'), Paragraph(children=[Text(text='hello mechmarket,', striked=False)], type='Paragraph'), Paragraph(children=[Text(text='Im here in a time of need for my other hobby. i have too many so this is to fund one of them.', striked=False), <Break.LineBreak: 'LineBreak'>, Text(text='I  have here a Peacock green matrix 1.2og with a linear setup: pom plate  linjars and FFFF linears all lubed and filmed(linjars not filmed) with  205g0.', striked=False)], type='Paragraph'), Paragraph(children=[Text(text='tactile version is dark ambers lubed and filmed with 205g0.', striked=False)], type='Paragraph'), Paragraph(children=[Text(text='im looking to get around 1100 Euros + shipping for these together.', striked=False), <Break.LineBreak: 'LineBreak'>, Text(text='Comes without the keycap ofc.', striked=False), Text(text='Hopefully within the EU. US is possible but i wont guarantee anythi